### Resource Used :

- https://www.youtube.com/watch?v=a2A_CxrH3Ts
- https://www.youtube.com/watch?v=vvjcGWnb0Os
- https://www.youtube.com/watch?v=hLzEHsUSHq4


In [3]:
pip install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.199.0
    Uninstalling sagemaker-2.199.0:
      Successfully uninstalled sagemaker-2.199.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'databricks/dolly-v2-3b',
	'SM_NUM_GPUS': json.dumps(1),
    'HUGGING_FACE_HUB_TOKEN': "REPLACE WITH YOUR TOKEN"
}

assert hub['HUGGING_FACE_HUB_TOKEN'] != '<REPLACE WITH YOUR TOKEN>', "You have to provide a token."

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri('huggingface',region='us-east-2',version='1.3.1'),
    #image_uris.retrieve(framework='huggingface',region='us-east-2',version='4.4.2',image_scope='training',base_framework_version='tensorflow2.4.1')
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.2xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "My name is Julien and I like to",
})

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
----------!

[{'generated_text': "My name is Julien and I like to write about technology, gaming, and pop culture.\n\nI'm a huge fan of the Witcher series by CD Projekt Red. I've been playing the latest game on PC, The Witcher 3: Wild Hunt, since it came out in May 2015. I also enjoy reading about new games and playing old games."}]

In [2]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
print(get_huggingface_llm_image_uri("huggingface",version="1.1.0"))

763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04


In [3]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
print(get_huggingface_llm_image_uri("huggingface",version="1.3.1",region='us-east-2'))

763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.1-tgi1.3.1-gpu-py310-cu121-ubuntu20.04


##### See the End-Point below

In [9]:
endpoint_name = 'huggingface-pytorch-tgi-inference-2023-12-14-09-27-33-391'

### The below code is to develop a serverless server , for which I have used "AWS Lambda" tool

1. Open AWS Lambda
2. Click Function, Assign Function Name.
3. Change Runtime to "Python 3.10x"
4. In "Change Default Execution Role" select "Create a new role with basic Lambda permissions"
5. Click on Create the function
6. Copy Paste the below code,  
7. Click configurations -> environment variables -> Create New Environment Variable -> **Place yours:** **ENDPOINT_NAME** 
8. Go to configurations -> permissions -> click Role Name -> Add Permissions -> search for "Sagemaker" -> Check "AWS Sagemaker FullAccess" -> Add Permission.
9. Go to Lambda function panel, then click "Deploy".
10. Go to Lambda function panel, then click "Test".

In [15]:
import os
import json
import boto3
import csv
import io

ENDPOINT_NAME = os.environ["ENDPOINT_NAME"]
#runtime = boto3.client('runtime.sagemaker')
runtime = boto3.client('sagemaker-runtime')

def lambda_handler(event,context):  
    print("Received Event: "+ json.dumps(event))
    
    data = json.dumps(event)
    payload = data
    print(payload)
    
    response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
    ContentType = "application/json",
    Body = bytes(payload, 'utf-8'))
    
    print(response)
    result = json.loads(response["Body"].read().decode())
    print(result)
    
    return result



In [16]:
result = lambda_handler(event = {"inputs": "My name is Julien and I like to"})
result

Received Event: {"inputs": "My name is Julien and I like to"}
{"inputs": "My name is Julien and I like to"}
{'ResponseMetadata': {'RequestId': '8b003513-9d6a-428a-bcec-3fbd5199ac16', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8b003513-9d6a-428a-bcec-3fbd5199ac16', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Thu, 14 Dec 2023 10:08:14 GMT', 'content-type': 'application/json', 'content-length': '320', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f8eee95f250>}
[{'generated_text': "My name is Julien and I like to write about technology, gaming, and pop culture.\n\nI'm a huge fan of the Witcher series by CD Projekt Red. I've been playing the latest game on PC, The Witcher 3: Wild Hunt, since it came out in May 2015. I also enjoy reading about new games and playing old games."}]


[{'generated_text': "My name is Julien and I like to write about technology, gaming, and pop culture.\n\nI'm a huge fan of the Witcher series by CD Projekt Red. I've been playing the latest game on PC, The Witcher 3: Wild Hunt, since it came out in May 2015. I also enjoy reading about new games and playing old games."}]

In [19]:
result[0]["generated_text"]

"My name is Julien and I like to write about technology, gaming, and pop culture.\n\nI'm a huge fan of the Witcher series by CD Projekt Red. I've been playing the latest game on PC, The Witcher 3: Wild Hunt, since it came out in May 2015. I also enjoy reading about new games and playing old games."

### The below is the to create the RestAPI using "AWS API Gateway" and testing it using PostMan / Requests method.

#### Steps to be followed

1. Go to "AWS API Gateway"
2. Click on "Create API" -> REST API  -> Build -> Give Details (Keep API Endpoint type = 'Regional' for public use )
3. Create a Resource -> Give Resource Name
4. Click on Create Method -> Select "POST" in Method Type  -> Select "Lambda Function" in the Integration type.
5. In the same page choose the Lambda Function
6. Create "Method"
7. Deploy API



In [41]:
import requests

In [36]:
url = "https://7g41u4103k.execute-api.us-east-2.amazonaws.com/dolly3bllm_test/deploy_dolly3b"
myobj = json.dumps({'inputs':"My name is Julien and I like to"})

x = requests.request("POST",url,headers = {'content-type':"application/json"},data = myobj)

print(x.text)

[{"generated_text": "My name is Julien and I like to write about technology, gaming, and pop culture.\n\nI'm a huge fan of the Witcher series by CD Projekt Red. I've been playing the latest game on PC, The Witcher 3: Wild Hunt, since it came out in May 2015. I also enjoy reading about new games and playing old games."}]


Finally Sucess !! Have a Toffee !!

In [37]:
x

<Response [200]>

### Very Important. After completion of your trial or rough work on AWS Sagemaker, delete the endpoint created. Otherwise it cost you more charges.

In [40]:
predictor.delete_endpoint()  #To delete up all the resources that have been used for this deployment